In [1]:
from modules.data import Data
from modules.rag import resolve_context
from modules.utils import resumable, save_progress

from tqdm import tqdm

In [2]:
data = Data()
data_test = data.get_split("test")

2024-07-16 17:26:50.133 | INFO     | modules.data:load_data:18 - load dev data, len: 1000
2024-07-16 17:26:50.140 | INFO     | modules.data:load_data:18 - load test data, len: 1000
2024-07-16 17:26:50.176 | INFO     | modules.data:load_data:18 - load train data, len: 5000


In [3]:
context_template = """
产品名：{product_name}
条款：{clause}
"""

In [ ]:
@resumable(progress_file="./cache/progress_test.json")
def process_data_with_rag(data, context_template, start_idx=0, progress_file=None):
    for idx, item in tqdm(
        enumerate(data[start_idx:], start=start_idx), initial=start_idx, total=len(data)
    ):
        product_name, raw_context, query = item["产品名"], item["条款"], item["问题"]
        if len(raw_context) > 500:
            retrieved_nodes = resolve_context(
                idx=idx, raw_context=raw_context, query=query
            )
            context = ""
            for node_with_score in retrieved_nodes:
                context += node_with_score.node.text
        else:
            context = raw_context
        data[idx]["context"] = context_template.format(
            product_name=product_name, clause=context
        )

        # Save progress after each item is processed
        if progress_file:
            save_progress(progress_file, data, ensure_ascii=False, indent=4)

    return data


# 使用装饰器的函数
data_test_new = process_data_with_rag(data=data_test, context_template=context_template)